# 🌦️ WeatherWise – Starter Notebook

Welcome to your **WeatherWise** project notebook! This scaffold is designed to help you build your weather advisor app using Python, visualisations, and AI-enhanced development.

---




# 🌦️ WeatherWise – Starter Notebook

Welcome to your **WeatherWise** project notebook! This scaffold is designed to help you build your weather advisor app using Python, visualisations, and AI-enhanced development.

---




In [ ]:
# 🧪 Optional packages — uncomment if needed in Colab or JupyterHub
!pip install request
!pip install ipywidgets
!pip install matplotlib

## 📦 Setup and Configuration
Import required packages and setup environment.

In [2]:
import requests
import json
import re
from datetime import datetime, timedelta
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt

## 🌤️ Weather Data Functions

In [3]:
# Define get_weather_data() function here
def get_weather_data(location, format_type="text"):
    try:
        # Set URL based on format type
        if format_type.lower() == "json":
            url = f'https://wttr.in/{location}?format=j1'
        else:  # default to text
            url = f'https://wttr.in/{location}'

        response = requests.get(url)
        response.raise_for_status()

        if format_type.lower() == "json":
            # Parse and return the JSON response
            return response.json()
        else:
            # Print the text response as you were doing before
            print(response.text)
            return None

    except requests.exceptions.HTTPError as errh:
        print(f'HTTP Error: {errh}')
    except requests.exceptions.ConnectionError as errc:
        print(f'Connection Error: {errc}')
    except requests.exceptions.Timeout as errt:
        print(f'Timeout Error: {errt}')
    except requests.exceptions.RequestException as err:
        print(f'Error: {err}')
    except json.JSONDecodeError as jerr:
        if format_type.lower() == "json":
            print(f'JSON Decode Error: {jerr}')

    return None

## 📊 Visualisation Functions

In [4]:
# Define create_temperature_visualisation() and create_precipitation_visualisation() here
#create weather emojis-------------------------------------------------------

def get_weather_emoji(weather_description):
    """Return an emoji based on the weather description"""
    weather_lower = weather_description.lower()

    # Define specific keywords for each weather condition #list_dictionary
    rain_keywords = ['rain', 'drizzle', 'shower', 'thundery outbreaks', 'downpour', 'wet']
    snow_keywords = ['snow', 'blizzard', 'sleet', 'ice', 'hail', 'flurry']
    cloud_keywords = ['cloud', 'overcast', 'mist', 'fog', 'haze', 'partly cloudy', 'scattered clouds']
    sun_keywords = ['sunny', 'clear', 'fair', 'hot', 'bright', 'clear skies', 'clear weather']
    wind_keywords = ['wind', 'gale', 'storm', 'breeze', 'gust']

    # Rain conditions
    if any(word in weather_lower for word in rain_keywords):
        if 'thunder' in weather_lower or 'storm' in weather_lower or 'thundery outbreaks' in weather_lower:
            return '⛈️'  # Thunderstorm
        elif 'light' in weather_lower or 'patchy' in weather_lower:
            return '🌦️'  # Sun behind rain cloud
        else:
            return '🌧️'  # Rain cloud

    # Snow conditions
    elif any(word in weather_lower for word in snow_keywords):
        if 'light' in weather_lower or 'patchy' in weather_lower:
            return '🌨️'  # Snow cloud
        else:
            return '❄️'  # Snowflake

    # Cloud conditions
    elif any(word in weather_lower for word in cloud_keywords):
        if 'fog' in weather_lower or 'mist' in weather_lower:
            return '🌫️'  # Fog
        elif 'overcast' in weather_lower:
            return '☁️'  # Cloudy weather
        elif 'partly' in weather_lower or 'scattered' in weather_lower:
            return '⛅'  # Sun behind cloud
        else:
            return '☁️'  # Cloud

    # Clear conditions (sunny/fair weather)
    elif any(word in weather_lower for word in sun_keywords):
        return '☀️'  # Sun

    # Wind conditions
    elif any(word in weather_lower for word in wind_keywords):
        return '💨'  # Wind blowing

    # Default (unknown or undefined weather)
    else:
        return '🌡️'  # Thermometer (default)


#create temperature visualisation using JSON--------------------------------

def create_temperature_visualisation(location, target_date, return_data=False):
    weather_data = get_weather_data(location, format_type="json")
    if not weather_data:
        print("Unable to get weather data")
        return None if return_data else None

    target_date_str = target_date.strftime("%Y-%m-%d")

    matched_day = None
    for day in weather_data['weather']:
        if day['date'] == target_date_str:
            matched_day = day
            break

    if matched_day:
        date_obj = datetime.strptime(matched_day['date'], "%Y-%m-%d")
        formatted_date = date_obj.strftime("%A, %d %B")

        min_temp = int(matched_day['mintempC'])
        max_temp = int(matched_day['maxtempC'])
        avg_temp = int(matched_day['avgtempC'])
        weather_desc = matched_day['hourly'][4]['weatherDesc'][0]['value']
        weather_emoji = get_weather_emoji(weather_desc)

        if not return_data:
            print("\n=== WEATHER CONDITION ===")
            print(f"Date: {formatted_date}")
            print(f"Weather: {weather_emoji} {weather_desc}")
            print(f"Minimum Temperature: {min_temp}°C")
            print(f"Maximum Temperature: {max_temp}°C")
            print(f"Average Temperature: {avg_temp:.1f}°C")
            print("\n" + "=" * 50)
        else:
            return {
                "date": formatted_date,
                "weather": f"{weather_emoji} {weather_desc}",
                "min_temp": min_temp,
                "max_temp": max_temp,
                "avg_temp": avg_temp
            }
    else:
        if not return_data:
            print(f"No weather forecast available for {target_date.strftime('%A, %d %B')}")
        return None if return_data else None

In [5]:
# Define create_precipitation_visualisation here
def create_precipitation_visualisation(weather_data, location, day_index=0, output_type='display'):
    # Ambil data hourly dari hari yang dipilih (biasanya 0,1,2)
    hourly_data = weather_data['weather'][day_index]['hourly']

    # Ambil jam (biasanya 8 data per hari: 0,3,6,9,12,15,18,21)
    jam = [int(hour['time']) // 100 for hour in hourly_data]
    precip = [float(hour['precipMM']) for hour in hourly_data]
    chance = [int(hour['chanceofrain']) for hour in hourly_data]

    import matplotlib.pyplot as plt

    fig, ax1 = plt.subplots(figsize=(10,5))

    ax1.plot(jam, precip, 'b-', marker='o', label='Precipitation (mm)')
    ax1.set_xlabel('Hour (24h)')
    ax1.set_ylabel('Precipitation (mm)', color='b')
    ax1.tick_params(axis='y', labelcolor='b')

    ax2 = ax1.twinx()
    ax2.plot(jam, chance, 'r--', marker='x', label='Chance of Rain (%)')
    ax2.set_ylabel('Chance of Rain (%)', color='r')
    ax2.tick_params(axis='y', labelcolor='r')

    plt.title(f'Current Graphical Representation of Precipitation and Rainfall Probability at {location})')
    plt.xticks(jam)
    fig.tight_layout()

    plt.show()

    plt.close()

## 🤖 Natural Language Processing

In [7]:
# Define parse_weather_question() and generate_weather_response() here
def parse_weather_question(question, location):
    # Definisikan pola regex untuk mendeteksi waktu
    time_pattern = r"(today|tomorrow|after[-\s]?tomorrow|day[-\s]?after[-\s]?tomorrow)"

    # Daftar aktivitas outdoor dan indoor
    outdoor_activities = r"(run|swim|sports|tennis|cycling|hiking|jogging|fishing|camping|skiing|sailing|kayaking|gardening)"
    indoor_activities = r"(cooking|reading|studying|watching movies|baking|painting|knitting|gaming|sleeping)"

    # Cari waktu dalam pertanyaan
    time_match = re.search(time_pattern, question, re.IGNORECASE)

    # Cari semua aktivitas outdoor
    outdoor_matches = re.findall(outdoor_activities, question, re.IGNORECASE)
    # Cari semua aktivitas indoor
    indoor_matches = re.findall(indoor_activities, question, re.IGNORECASE)

    # Default waktu ke "today" jika tidak ditemukan
    time = "today"
    activities = []  # Menyimpan aktivitas yang ditemukan
    activity_types = []  # Menyimpan jenis aktivitas (indoor atau outdoor)

    if time_match:
        time = time_match.group(1).lower()  # Ambil waktu (today, tomorrow, after-tomorrow)

    # Tambahkan aktivitas outdoor yang ditemukan
    if outdoor_matches:
        activities.extend(outdoor_matches)
        activity_types.extend(["Outdoor"] * len(outdoor_matches))

    # Tambahkan aktivitas indoor yang ditemukan
    if indoor_matches:
        activities.extend(indoor_matches)
        activity_types.extend(["Indoor"] * len(indoor_matches))

    # Tentukan tanggal berdasarkan waktu yang disebutkan
    target_date = datetime.now()

    if time == "today":
        # Just use the current date without modifying it.
        target_date = target_date.replace(hour=0, minute=0, second=0, microsecond=0)  # Set to midnight of today
    elif time == "tomorrow":
        target_date += timedelta(days=1)
        target_date = target_date.replace(hour=0, minute=0, second=0, microsecond=0)  # Set to midnight of tomorrow
    elif time in ["after tomorrow", "day after tomorrow"]:
        target_date += timedelta(days=2)
        target_date = target_date.replace(hour=0, minute=0, second=0, microsecond=0)  # Set to midnight of the day after tomorrow

    # Panggil fungsi untuk menampilkan cuaca berdasarkan lokasi dan tanggal
    if location:
        weather = create_temperature_visualisation(location, target_date, return_data=True)
        if not weather:
            print("No Data..")
            return

        if activities:
            for activity, activity_type in zip(activities, activity_types):
                response = generate_weather_response(
                    location=location,
                    time = time,
                    date_str=target_date.strftime('%A, %d %B %Y'),
                    activity=activity,
                    activity_type=activity_type,
                    weather_desc=weather["weather"],
                    min_temp=weather["min_temp"],
                    max_temp=weather["max_temp"],
                    avg_temp=weather["avg_temp"]
                )
                print(response)
        else:
            print("No specific activity mentioned.")
    else:
        print("Please provide a valid location.")


  #get_weather_response----------------------------------------------------------
def generate_weather_response(location, time ,date_str, activity, activity_type, weather_desc, min_temp, max_temp, avg_temp):
    desc = weather_desc.lower()
    activity_suggestion = ""

    bad_weather = any(keyword in desc for keyword in ['rain', 'thunder', 'fog', 'mist', 'shower','snow', 'drizzle', 'downpour', 
                                                      'blizzard', 'sleet', 'ice', 'hail', 'flurry'])
    if activity_type.lower() == "outdoor":
        if bad_weather or avg_temp < 15:
            activity_suggestion = (f"The weather is bad or it's cold ({avg_temp}°C), so outdoor activities like {activity.capitalize()} "
                               "are not recommended. It's better to stay indoors.")
        elif 15 <= avg_temp <= 30:
            activity_suggestion = (f"The weather and temperature ({avg_temp}°C) are comfortable, perfect for outdoor activities like "
                               f"{activity.capitalize()}.")
        else:  # avg_temp > 30 and weather is good
            activity_suggestion = (f"It's very hot outside ({avg_temp}°C). Even though the weather is good, be careful doing outdoor activities "
                               f"like {activity.capitalize()}. Indoor activities are recommended.")
    else:  # indoor activity
        if bad_weather or avg_temp < 15:
            activity_suggestion = (f"The weather is bad or it's cold ({avg_temp}°C), so indoor activities like {activity.capitalize()} "
                               "are the best choice.")
        else:  # good weather and comfortable/hot temperature
            activity_suggestion = (f"The weather is good and temperature is {avg_temp}°C. Indoor activities like {activity.capitalize()} are fine, "
                               "but you can also try outdoor activities like Jogging or Basketball.")
        
    output = (
                "\n" + "-" * 53
                +"\n" + " " * 20 + "ACTIVITY SUGGESTION"
                +"\n" + "-" * 53
                +"\n" + f"📍 Location   : {location}"
                +"\n" + f"📅 Date       : ({time.capitalize()}) {date_str}"
                +"\n" + f"🏃 Activity   : ({activity_type.capitalize()})-{activity.capitalize()}"
                +"\n" + f"🌤️ Weather    : {weather_desc}"
                +"\n" + f"🌡️ Temperature: Min {min_temp}°C | Max {max_temp}°C | Avg {avg_temp}°C"
                +"\n"
                +"\n" + "✅ Suggestion:"
                +"\n" + f"{time.capitalize()}, " +activity_suggestion
                +"\n" +"-" * 53 + "\n"
                )
    
    return output


## 🧭 User Interface

In [11]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Variabel global widget dan user info
user_info = {'name': '', 'location': ''}
name_input = widgets.Text(description='🤖',layout= widgets.Layout(width='300px'), placeholder='Hi! Please enter your name')
location_input = widgets.Text(description='🤖',layout= widgets.Layout(width='300px'),placeholder='Which location would you check?')
btn_start = widgets.Button(description='Start', layout=widgets.Layout(margin='10px 0 0 90px'))
input_box = widgets.VBox()
output = widgets.Output()

def main():
    def show_menu():
        with output:
            clear_output()
            name = user_info['name']
            location = user_info['location']

            print(f'🤖 Hello! {name}, Welcome to Weather Wise. How can I help you at {location}?')
            print("-" * 53)
            print('Please enter your choice (1 - 6):')
            print(f'1. Check Current Weather & View Weather Forecast at {location}')
            print(f'2. Check Current Weather Hourly at {location}')
            print(f'3. Activities Suggestions at {location}')
            print(f'4. Weather prediction assistant at {location}')
            print('5. Check Other Location')
            print('6. Exit Program')

            input_choose = widgets.Text(description="🤖", placeholder='Enter your choice (1-6)')
            btn_choose = widgets.Button(description="Check",layout=widgets.Layout(margin='10px 0 0 90px'))
            

            def on_menu_select(b):
                with output:
                    clear_output()
                    pilihan = input_choose.value.strip()
                    if pilihan == '1':
                        get_weather_data(location)
                    elif pilihan == '2':
                        data_cuaca = get_weather_data(location, format_type="json")
                        if data_cuaca:
                            create_precipitation_visualisation(data_cuaca,location)
                        else:
                            print("Failed to download..")
                    elif pilihan == '3':
                        question_text = widgets.Text(description="Question:", placeholder='ex: How abour running today?')
                        submit_q = widgets.Button(description="Submit", layout=widgets.Layout(margin='10px 0 0 90px'))

                        def on_submit_q(btn):
                            with output:
                                clear_output()
                                question = question_text.value
                                parse_weather_question(question, location)
                                show_back_button()

                        submit_q.on_click(on_submit_q)
                        display(widgets.Label("What Activities & When (Today, Tomorrow, Day After Tomorrow)?"))
                        display(question_text, submit_q)
                        return
                    elif pilihan == '4':
                        print("🧠 Asisten prediksi cuaca... [placeholder]")
                    elif pilihan == '5':
                        new_location = widgets.Text(description='🤖', placeholder='Enter New Location')
                        update_btn = widgets.Button(description="Update Location", layout=widgets.Layout(margin='10px 0 0 90px'))

                        def on_update_location(btn):
                            loc = new_location.value.strip()
                            with output:
                                clear_output()
                                if not loc:
                                    print("⚠️ Please enter a location before updating.")
                                else:
                                    user_info['location'] = loc
                                    print(f"📍 Your New Location: {user_info['location']}")
                                show_menu()

                        update_btn.on_click(on_update_location)
                        display(new_location, update_btn)
                        return
                    elif pilihan == '6':
                        # lalu reset
                        user_info['name'] = ''
                        user_info['location'] = ''
                        name_input.value = ''
                        location_input.value = ''
                        input_box.children = []
                        clear_output(wait=True)
                        print(f"Thank you! Have a nice day! {name}🌸")
                        return
                    else:
                        print("\033[1;31mInvalid choice. Please type 1-6 as a number!\033[0m")

                    show_back_button()

            def show_back_button():
                back_button = widgets.Button(description="Back to Main Menu")

                def on_back(b):
                    with output:
                        clear_output()
                        show_menu()

                back_button.on_click(on_back)
                display(back_button)

            display(input_choose, btn_choose)
            btn_choose.on_click(on_menu_select)

    def on_start(b):
        user_info['name'] = name_input.value.strip()
        user_info['location'] = location_input.value.strip() #strip is trim
        input_box.children = []
        with output:
            clear_output()
        show_menu()

    btn_start.on_click(on_start)

## 🧩 Main Application Logic

In [12]:
# Tie everything together here

if __name__ == "__main__":
    print("=" * 53)
    input_box.children = [widgets.HTML('<h2 style="color:blue; margin-left:80px;">🌤️ Weather Wise</h2>'), name_input, location_input, btn_start]
    display(input_box, output)
    print("=" * 53)

    main()


Output()

## 🧪 Testing and Examples

In [ ]:
# Include sample input/output for each function

## 🗂️ AI Prompting Log (Optional)
Add markdown cells here summarising prompts used or link to AI conversations in the `ai-conversations/` folder.